In [1]:
from import_library import *
from model_stack import model_list

In [2]:
print("CPU count: {}".format(multiprocessing.cpu_count()))

CPU count: 8


In [3]:
data_type ='var' #mat, csv, mix

# dataloader
clinical_data = pd.read_csv('./data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]', 'HADS_A焦慮程度', 'HADS_D憂鬱程度']]
raw_data_path = './data/mdALFF matrix excel/' #raw_data_path = './data/image data/dFC matrix matlab/'
if data_type!='mix':
    next_path =f'mdALFF_{data_type}.csv' #next_path ='C0005FC_z.mat't'
else:
    next_path = ['mdALFF_mean.csv', 'mdALFF_var.csv']
def loader_(data_path = None, data_type='mat' ,mat_dtype='FCM_mean'):
    if '.mat' in data_path:
        mat = scipy.io.loadmat(data_path)
        get_value = mat[mat_dtype ]
        return get_value.shape

    else:
        pd_tb = pd.read_csv(data_path)
        pd_patient_index = pd_tb[pd_tb.columns[0]]
        pd_patient_value = pd_tb[pd_tb.columns[1::]]
        return pd_patient_index, pd_patient_value

In [4]:
from itertools import compress
# feature selection process
rank_threshold = 60
def feature_filter(patient_value, load_path = None,outcome_name = None, data_type = None):
    feature_rank_df = pd.read_csv('{}/{}_{}_{}.csv'.format(load_path, outcome_name,data_type, 'alff'), index_col=False)
    filter_feature_list = [feature_rank_df['rank'].values<=rank_threshold][0]
    feature_columns = patient_value.columns.tolist()
    patient_value = patient_value[list(compress(feature_columns, filter_feature_list))]
    return patient_value

In [5]:
grid =False
feature_selection_df_savepath = './feature_selection_result'
grid_stack = []
result_dict={'outcome':[], 'model':[], 'Reg:mae(train)':[], 'Reg:mae(valid)': [], 'Reg:mse(train)':[], 'Reg:mse(valid)': [], 'Reg:mape(train)':[], 'Reg:mape(valid)': [], }
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()
        # classification : AD8 [0-1, >=2]{0,1}, MOCA [<=23, >=24]{0,1}
        # outcome_name = 'MOCA客觀認知測驗分數[<=23,>=24]' #AD8主觀認知障礙[0-1,>=2], MOCA客觀認知測驗分數[<=23,>=24]
        model_stack = model_list(grid=grid)
        kfold = KFold(n_splits=4)
        for outcome_name in ["AD8主觀認知障礙[0-1,>=2]", "MOCA客觀認知測驗分數[<=23,>=24]"]:
            for model in model_stack:
                if model in ['MLPR', 'XBGR', 'LGBMR']:
                    result_dict['outcome'].append(outcome_name)
                    result_dict['model'].append(model)
                    label_ = clinical_data[outcome_name]
                    if 'MOCA' in outcome_name:
                        label_s = [1 if (i>=24)  else 0  for i in list(label_)]
                    elif 'AD8' in outcome_name:
                        label_s = [1 if i>=2 else 0 for i in list(label_)]
                        label_ = label_+1
                    if data_type!='mix':
                        _, patient_value = loader_(data_path = os.path.join(raw_data_path,next_path))
                    else:
                        _, patient_value = loader_(data_path = os.path.join(raw_data_path,next_path), data_type='mix')
                    # patient_value_index = clinical_data['subj']
                    # patient_value = feature_filter(patient_value, load_path = feature_selection_df_savepath, outcome_name = outcome_name, data_type = data_type, )
                    # X_train, X_test, Y_label, _  = train_test_split(patient_value, list(label_s) , stratify=list(label_s), random_state=123)
                    # y_train, y_test, _, _ = train_test_split(list(label_), list(label_s) , stratify=list(label_s), random_state=123)
                    # X_train, X_valid, _, _ = train_test_split(X_train, list(Y_label) , stratify=list(Y_label), test_size=0.5, random_state=123)
                    # y_train, y_valid, _, _ = train_test_split(y_train, list(Y_label) , stratify=list(Y_label), test_size=0.5,random_state=123)
                    # print("Feature Number: {}".format(X_train.shape[1]))
                    # if grid==False:
                    #     print("ALFF Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                    #     print("--Start Training {} model--".format(model))
                    #     if model in 'MLPR':
                    #         model = model_stack[model].fit(X_train, y_train)
                    #     else:
                            
                    #         model = model_stack[model].fit(np.array(X_train), np.array(y_train), eval_set=[[np.array(X_valid), np.array(y_valid)]], eval_metric="mape", verbose=False)
                    #     train_pred =  model.predict(X_train)
                        
                    #     valid_pred =  model.predict(X_valid)
                    # else:
                    #     print("ALFF Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                    #     print("--Start Training {} model--".format(model))
                    #     GridModel = GridSearchCV(model_stack[model][0],model_stack[model][1], cv=kfold, n_jobs=-1, verbose=-1, refit = 'neg_mean_absolute_percentage_error', scoring="neg_mean_absolute_percentage_error",
                    #                              return_train_score=True)
                    #     GridModel.fit(np.array(X_train).astype(np.float16), np.array(y_train).astype(np.int8))
                    #     param_grid = model_stack[model][1]
                    #     # print("Grid Metric", GridModel.cv_results_.keys())
                    #     # for res_ in GridModel.cv_results_:
                    #     #     if 'test_score' in res_:
                    #     #         print(res_,GridModel.cv_results_[res_])
                    #     # print("Scoring", GridModel.best_score_)
                    #     print('Best parameter : [{}]\n'.format(GridModel.best_params_))
                    #     train_pred =  GridModel.predict(X_train)
                    #     valid_pred =  GridModel.predict(X_valid)
                    #     print("Data Length. Training : {} || Validation: {}".format(len(y_train), len(y_valid)))
                    #     print("pass",model)
                    #     grid_stack.append(GridModel)
                    
                    # result_dict['Reg:mae(train)'].append(round(metrics.mean_absolute_error(y_train,train_pred), 5))
                    # result_dict['Reg:mae(valid)'].append(round(metrics.mean_absolute_error(y_valid,valid_pred), 5))
                    # result_dict['Reg:mse(train)'].append(round(metrics.mean_squared_error(y_train,train_pred),5))
                    # result_dict['Reg:mse(valid)'].append(round(metrics.mean_squared_error(y_valid,valid_pred),5))
                    # result_dict['Reg:mape(train)'].append(round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
                    # result_dict['Reg:mape(valid)'].append(round(metrics.mean_absolute_percentage_error(y_valid,valid_pred),5))

                    # print('Train MAE: ', round(metrics.mean_absolute_error(y_train,train_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_train,train_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
                    # print('Valid MAE: ', round(metrics.mean_absolute_error(y_valid,test_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_valid,test_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_valid,test_pred),5))
                    # print('Test MAE: ', round(metrics.mean_absolute_error(y_test,test_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_test,test_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_test,test_pred),5))
                    print('----------------------------------------------------')
                # print('\n')
                

Feature Number: 61
ALFF Outcome: AD8主觀認知障礙[0-1,>=2][var] - cut rank point: 60 - GridSearch: False
--Start Training MLPR model--
----------------------------------------------------
Feature Number: 61
ALFF Outcome: AD8主觀認知障礙[0-1,>=2][var] - cut rank point: 60 - GridSearch: False
--Start Training XBGR model--
----------------------------------------------------
Feature Number: 61
ALFF Outcome: AD8主觀認知障礙[0-1,>=2][var] - cut rank point: 60 - GridSearch: False
--Start Training LGBMR model--
----------------------------------------------------
Feature Number: 58
ALFF Outcome: MOCA客觀認知測驗分數[<=23,>=24][var] - cut rank point: 60 - GridSearch: False
--Start Training MLPR model--
----------------------------------------------------
Feature Number: 58
ALFF Outcome: MOCA客觀認知測驗分數[<=23,>=24][var] - cut rank point: 60 - GridSearch: False
--Start Training XBGR model--
----------------------------------------------------
Feature Number: 58
ALFF Outcome: MOCA客觀認知測驗分數[<=23,>=24][var] - cut rank point: 60 -

In [6]:
for i in grid_stack:
    print(i.best_params_)
    print('----------------------------------------------------------------------------------------------------------------------------------------')

In [7]:
result_df = pd.DataFrame(result_dict) 
result_df.style.set_caption("AD8/MOCA - [ALFF - {} - Grid: {}]".format(data_type, grid))

,outcome,model,Reg:mae(train),Reg:mae(valid),Reg:mse(train),Reg:mse(valid),Reg:mape(train),Reg:mape(valid)
0,"AD8主觀認知障礙[0-1,>=2]",MLPR,1.789890,1.763410,5.736110,5.121500,0.740760,0.720580
1,"AD8主觀認知障礙[0-1,>=2]",XBGR,0.000260,1.574410,0.000000,4.390700,0.000140,0.655900
2,"AD8主觀認知障礙[0-1,>=2]",LGBMR,1.858480,1.835690,5.273670,4.648710,1.037240,1.020630
3,"MOCA客觀認知測驗分數[<=23,>=24]",MLPR,3.489350,2.901190,16.783460,12.054100,0.139820,0.114830
4,"MOCA客觀認知測驗分數[<=23,>=24]",XBGR,0.000270,2.530370,0.000000,8.876770,0.000010,0.104040
5,"MOCA客觀認知測驗分數[<=23,>=24]",LGBMR,3.078040,2.750000,12.888660,10.476260,0.126990,0.113440
